<a href="https://colab.research.google.com/github/ikanx101/G-Colab/blob/main/home_tester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# sentiment analysis
# home tester club

library(dplyr)
library(tidyr)
library(tidytext)

In [6]:
load("scrape TS.rda")
str(data)
stop = readLines("https://raw.githubusercontent.com/stopwords-iso/stopwords-id/master/stopwords-id.txt")

'data.frame':	165 obs. of  4 variables:
 $ produk   : chr  "Tropicana Slim Sweet Orange" "Tropicana Slim Sweet Orange" "Tropicana Slim Sweet Orange" "Tropicana Slim Sweet Orange" ...
 $ rate     : chr  "4,9 dari 165 Review & Komentar" "4,9 dari 165 Review & Komentar" "4,9 dari 165 Review & Komentar" "4,9 dari 165 Review & Komentar" ...
 $ deskripsi: chr  "Minuman serbuk instan rasa jeruk yang tinggi serta & 7 vitamin" "Minuman serbuk instan rasa jeruk yang tinggi serta & 7 vitamin" "Minuman serbuk instan rasa jeruk yang tinggi serta & 7 vitamin" "Minuman serbuk instan rasa jeruk yang tinggi serta & 7 vitamin" ...
 $ komen    : chr  "Minuman serbuk rasa jeruk yang manisnya cocok untuk penderita diabetes aman pokoknya" "Saya adalah orang yang mengutamakan kesehatan, pingin manis tapi tetap segar saya biasa gunakan tropicana slim "| __truncated__ "Suka banget dengan minuman ini, manisnya pas. Buat aku yg nggak suka manis dan menghindarinya yg namanya kalori"| __truncated__ "Minumam ini co

In [7]:
data_komen = data.frame(
  id = c(1:length(data$komen),
  komen = tolower(data$komen))
)